# Greenland Ice Surface Temperature, Surface Albedo, and Water Vapor from MODIS Comparison Tool
This tool compares observations of Greenland Ice Sheet surface temperature, albedo, and water vapor to estimates from climate model reanalysis. The observations are compiled in a multilayer data product called Multilayer Greenland Ice Surface Temperature, Surface Albedo, and Water Vapor from MODIS, Version 1 (MODGRNLD; https://nsidc.org/data/MODGRNLD). This dataset is available at monthly and daily resolutions. This tool compares the observations to modeled variables from the Modern-Era Retrospective analysis for Research and Applications, Version 2 (MERRA-2; https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2). This tool uses monthly data from both the MODGRNLD and MERRA-2 data products for comparison. Additional information about the MODGRNLD data product and comparisons with MERRA-2 can be found in the reference paper for this tool (Hall et al., 2018).
This tool interpolates MERRA-2 data to the higher-resolution MODGRNLD observations, creates map-view plots and a histogram of the differences. The user is able to download both plots.
The plots generated by the comparison between datasets show:
1. **MODGRNLD Data**: Number of days used in monthly average calculation based on the locations of collected satellite observations.
1. **Observed variable**: Measurements of the surface variable from MODGRNLD.
1. **Modeled variable**: Estimate of the surface variable from MERRA-2.
1. **Difference**: Difference between observed (MODGRNLD) and modeled (MERRA-2) surface variable.
1. **Statistics and Histogram**: Statistics and histogram of the differences between MODGRNLD and MERRA-2 surface variable.
### References
Hall, D., Cullather, R., DiGirolamo, N., Comiso, J., Medley, B., & Nowicki, S. (2018). A Multilayer Surface Temperature, Surface Albedo, and Water Vapor Product of Greenland from MODIS. Remote Sensing, 10(4), 555–17. http://doi.org/10.3390/rs10040555

Tool Info:
[MODGRNLD Data](#MODGRNLD_cell)
[MERRA-2 Data](#MERRA2_cell)

<a id = 'howto_cell'></a>
## How to Use this Tool
The user can define the parameters that are used throughout the model-observation comparison tool. Here are the parameters that the user can choose to change:

1. **Comparison Year**: Year for model-observation comparison.
3. **Comparison Month**: Month for model-observation comparison.
4. **MODGRNLD Data Version**: Version Number for MODGRNLD data to be used in model-observation comparison.

In [1]:
import os, sys, shutil, json, stat
from hublib.ui import Download, Dropdown, Togglebuttons, Modal
import ipywidgets as widgets 

import numpy as np

import IPython

sys.path.append('data')
import nsidc_download_tools
import data_comparison_tools

import time

##Used for Earthdata login##
#Implement Later#
#import gimpfunc.NASALogin as NASALogin
#import panel
#panel.extension()

<IPython.core.display.Javascript object>

In [2]:
def set_user_options():
    global comp_years, version_options
    
    info_file = open("/data/groups/ghub/tools/modgrnld/file_info.txt",'r')
    
    comp_years = list()
    for line in info_file.readlines():
        filename = line.split('/')[-1]
        file_components = filename.split('.')
        if file_components[1][0:4] not in comp_years and file_components[1][0:4].isnumeric():
            comp_years.append(file_components[1][0:4])
            
    info_file.close()
    
    info_file = open("/data/groups/ghub/tools/modgrnld/file_info.txt",'r')
    
    versions = list()
    for line in info_file.readlines():
        filename = line.split('/')[-1]
        file_components = filename.split('.')
        if file_components[3][0]=='v':
            if file_components[4].isnumeric() and (file_components[3]+'.'+file_components[4]) not in versions:
                versions.append(file_components[3]+'.'+file_components[4])
            elif not file_components[4].isnumeric() and file_components[3] not in versions:
                versions.append(file_components[3])
        elif file_components[4][0]=='v':
            if file_components[5].isnumeric() and (file_components[4]+'.'+file_components[5]) not in versions:
                versions.append(file_components[4]+'.'+file_components[5])
            elif not file_components[5].isnumeric() and file_components[4] not in versions:
                versions.append(file_components[4])
    version_options = versions
    
    info_file.close()

def set_months(comparison_year):
    comparison_year = int(comparison_year)
    
    info_file = open("/data/groups/ghub/tools/modgrnld/file_info.txt","r")
    
    MODGRNLD_filename_template = 'MODGRNLD.{:4d}'.format(comparison_year)
    
    month_options = list()
    for line in info_file.readlines():
        if MODGRNLD_filename_template not in line:
            continue
        filename = line.split('/')[-1]
        file_components = filename.split('.')
        if file_components[1][4:6] not in month_options:
            month_options.append(file_components[1][4:6])
            
    info_file.close()
    
    month_names = list()
    for m in month_options:
        month_names.append(comp_months_names[int(m)-1])
    
    return month_names
                
                
def set_versions(comparison_year, comparison_month):
    comparison_year = int(comparison_year)
    comparison_month = int(comparison_month)
    
    info_file = open("/data/groups/ghub/tools/modgrnld/file_info.txt","r")
    
    MODGRNLD_filename_template = 'MODGRNLD.{:4d}{:02d}.monthly'.format(comparison_year, comparison_month)
    
    comp_versions = list()
    for line in info_file.readlines():
        if MODGRNLD_filename_template not in line:
            continue
        filename = line.split('/')[-1]
        file_components = filename.split('.')
        if file_components[3][0]=='v':
            if file_components[4].isnumeric() and (file_components[3]+'.'+file_components[4]) not in comp_versions:
                comp_versions.append(file_components[3]+'.'+file_components[4])
            elif not file_components[4].isnumeric() and file_components[3] not in comp_versions:
                comp_versions.append(file_components[3])
        elif file_components[4][0]=='v':
            if file_components[5].isnumeric() and (file_components[4]+'.'+file_components[5]) not in comp_versions:
                comp_versions.append(file_components[4]+'.'+file_components[5])
            elif not file_components[5].isnumeric() and file_components[4] not in comp_versions:
                comp_versions.append(file_components[4])
                
    return comp_versions

In [3]:
def set_variables():
    global comparison_scope, comparison_variable, comparison_year, comparison_month, comparison_version, file_number, dataDir, MERRA_url_template, MERRA_filename_template, debug_plots, mask_colors
    # Year/month over which to do comparison
    comparison_scope = "Monthly" #Daily or Monthly data 
    comparison_variable = user_variable_choice.value
    comparison_year = int(user_year_choice.value)
    comparison_month = int(comp_months[user_month_choice.value])
    comparison_version = user_version_choice.value.strip()
    
    file_number = 0
    if comparison_year >= 1980 and comparison_year < 1992:
        file_number = 100
    elif comparison_year >= 1992 and comparison_year < 2001:
        file_number = 200
    elif comparison_year >= 2001 and comparison_year < 2011:
        file_number = 300
    elif comparison_year >= 2011:
        file_number = 400
    
    # GHub data directory
    dataDir = '/data/groups/ghub/tools/modgrnld'

    # MERRA data to download from GES DISC
    if comparison_variable=='Surface Temperature':
        if comparison_scope == 'Daily':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2SDNXSLV.5.12.4/{:4d}/{:02d}'
            MERRA_filename_template = 'MERRA2_{:3d}.statD_2d_slv_Nx.{:4d}{:02d}{:02d}.nc4'
        elif comparison_scope == 'Monthly':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2_MONTHLY/M2SMNXSLV.5.12.4/{:4d}'
            MERRA_filename_template = 'MERRA2_{:3d}.statM_2d_slv_Nx.{:4d}{:02d}.nc4'
    elif comparison_variable=='Albedo':
        if comparison_scope == 'Daily':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2T1NXRAD.5.12.4/{:4d}/{:02d}'
            MERRA_filename_template = 'MERRA2_{:3d}.tavg1_2d_rad_Nx.{:4d}{:02d}{:02d}.nc4'
        elif comparison_scope == 'Monthly':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2_MONTHLY/M2TMNXRAD.5.12.4/{:4d}'
            MERRA_filename_template = 'MERRA2_{:3d}.tavgM_2d_rad_Nx.{:4d}{:02d}.nc4'
    elif comparison_variable=='Water Vapor':
        if comparison_scope == 'Daily':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2I1NXINT.5.12.4/{:4d}/{:02d}'
            MERRA_filename_template = 'MERRA2_{:3d}.inst1_2d_int_Nx.{:4d}{:02d}{:02d}.nc4'
        elif comparison_scope == 'Monthly':
            MERRA_url_template = 'https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2_MONTHLY/M2IMNXINT.5.12.4/{:4d}'
            MERRA_filename_template = 'MERRA2_{:3d}.instM_2d_int_Nx.{:4d}{:02d}.nc4'

    # Plotting
    debug_plots = False
    mask_colors = np.array([[1.0, 1.0, 1.0],   # ocean
                            [0.5, 0.3, 0.1],   # land
                            [0.8, 0.8, 0.8],   # ice
                            [  1,   1,   1],   #
                            [  1,   1,   1]])  #
    
    #update progress bar
    progress_bar.value = 1
    
def print_job_info():
    text_output.clear_output()
    text_label.layout.visibility = 'visible'
    with text_output:
        display(text_label)
        print('Year Selected:  '+str(comparison_year))
        print('Month Selected: '+comp_months_names[comparison_month-1])
        print('MODGRNLD Data Version Selected: '+comparison_version+'\n')
    
    job_output_file = open('job_info.txt','r')
    with text_output:
        print('MODGRNLD Files Plotted:')
        print(job_output_file.readline(), end='')
        print(job_output_file.readline())
        print('MODGRNLD Data URL: '+'https://nsidc.org/data/MODGRNLD\n')
        print('MERRA-2 Files Plotted:')
    for line in job_output_file.readlines():
        with text_output:
            print(line,end='')
    with text_output:
        print('\nMERRA Data URL: '+MERRA_url_template.format(comparison_year,comparison_month))

In [4]:
def execute(b):
    exe_button.disabled = True
    text_label.layout.visibility = 'hidden'
    
    output_box1.layout.visibility = 'hidden'
    output_box2.layout.visibility = 'hidden'
    out_box.layout.visibility = 'hidden'
    
    modgrnld_label.layout.visibility = 'hidden'
    modgrnld_output.layout.visibility = 'hidden'
    
    surftemp_label.layout.visibility = 'hidden'
    surfacetemp_output.layout.visibility = 'hidden'
    
    stats_label.layout.visibility = 'hidden'
    stats_output.layout.visibility = 'hidden'
    
    ret = 0
    text_output.clear_output()
    modgrnld_output.clear_output()
    surfacetemp_output.clear_output()
    stats_output.clear_output()
    
    progress_bar.value = 0
    progress_bar.layout.visibility = 'visible'
    
    text_output.layout.visibility = 'visible'
    
    set_variables()
    with login_panel:
        data_comparison_tools.set_variables(comparison_scope, comparison_variable, comparison_year, comparison_month, comparison_version, file_number, dataDir, MERRA_url_template, MERRA_filename_template, debug_plots, mask_colors)
    progress_bar.value = 1
    
    with text_output:
        ret = data_comparison_tools.download_nsidc_data()
    if(ret!=0):
        progress_bar.layout.visibility = 'hidden'
        exe_button.disabled = False
        return None
    progress_bar.value = 2
    
    with text_output:
        #debugging timer
        global run_time
        start_time = time.time()
        ret = data_comparison_tools.download_merra_data()
        run_time = time.time()-start_time
    if(ret!=0):
        progress_bar.layout.visibility = 'hidden'
        exe_button.disabled = False
        return None
    progress_bar.value = 3
    
    with text_output:
        data_comparison_tools.read_grnld_icesheet_mask()
    progress_bar.value = 4
    
    with text_output:
        ret = data_comparison_tools.read_merra_data()
    if(ret!=0):
        progress_bar.layout.visibility = 'hidden'
        exe_button.disabled = False
        return None
    progress_bar.value = 5
    
    with text_output:
        ret = data_comparison_tools.read_modgrnld_data()
    if(ret!=0):
        progress_bar.layout.visibility = 'hidden'
        exe_button.disabled = False
        return None
    progress_bar.value = 6
    
    
    modgrnld_label.layout.visibility = 'visible'
    modgrnld_output.layout.visibility = 'visible'
    confirmation_button.layout.visibility = 'visible'
    cancel_button.layout.visibility = 'visible'
    with modgrnld_output:
        display(modgrnld_label)
        data_comparison_tools.plot_modgrnld_data()
        #display(widgets.HBox([confirmation_button,cancel_button]))
    execute_after_confirmation()
        
    
    
def execute_after_confirmation():
    confirmation_button.layout.visibility = 'hidden'
    cancel_button.layout.visibility = 'hidden'
    progress_bar.value = 7
    
    data_comparison_tools.interpolate_to_common_grid()
    progress_bar.value = 8
    
    with surfacetemp_output:
        if comparison_variable=='Surface Temperature':
            temp_text = " SURFACE TEMPERATURE DATA: "
        elif comparison_variable=='Albedo':
            temp_text = " ALBEDO DATA: "
        else: #Water Vapor
            temp_text = " WATER VAPOR DATA: "
        surftemp_label.value = f"<b>{temp_text}</b>"
        display(surftemp_label)
        data_comparison_tools.plot_surfacetemp_diff()
    progress_bar.value = 9
    
    with stats_output:
        display(stats_label)
        data_comparison_tools.calc_and_plot_stats()
    progress_bar.value = 10
    
    progress_bar.layout.visibility = 'hidden'
    
    print_job_info()
    
    output_box1.layout.visibility = 'visible'
    output_box2.layout.visibility = 'visible'
    out_box.layout.visibility = 'visible'
    
    surftemp_label.layout.visibility = 'visible'
    surfacetemp_output.layout.visibility = 'visible'
    
    stats_label.layout.visibility = 'visible'
    stats_output.layout.visibility = 'visible'
    
    exe_button.disabled = False
    
    with text_output:
        data_comparison_tools.clean_up()
    
    #TODO:
    #comparison of albedo and water vapor data from MERRA-2 files
    #also further use of VIIRS and AIRS data (planning needed)
    
def execute_cancelled(b):
    with modgrnld_output:
        print('Data Analysis Cancelled!')
    exe_button.disabled = False
    progress_bar.layout.visibility = 'hidden'
    confirmation_button.layout.visibility = 'hidden'
    cancel_button.layout.visibility = 'hidden'
    with text_output:
        data_comparison_tools.clean_up()

In [5]:
#if(os.path.isfile('year_options.txt') and os.path.isfile('version_options.txt')):
#    read_user_options()
#else:
#    set_user_options()
set_user_options()
    
comp_months_names = ('January','February','March','April','May','June','July','August','September','October','November','December')
comp_months = {'January':1,'February':2,'March':3,'April':4,'May':5,'June':6,'July':7,'August':8,'September':9,'October':10,'November':11,'December':12}

variable_options = ['Surface Temperature','Albedo','Water Vapor']

#default versions list
#default_versions = ['v01.1','test_version']

#TODO: dynamic version options based on year and month selection
#TODO: same for month options based on year selection
#def on_year_change(info, change): #change = new year selection value
#    user_month_choice.options = set_months(change)
#    user_month_choice.value = user_month_choice.options[0]
    #comp_versions = set_versions(change, comp_months[user_month_choice.value])
    #user_version_choice.options = comp_versions
    
    
# Widgets for retrieving user input data
user_variable_choice = Togglebuttons(
    name = 'Variable Selection',
    value = variable_options[0],
    options = variable_options,
    width = '25%')

user_year_choice = Dropdown(
    name = 'Comparison Year',
    value = comp_years[0],
    options = comp_years,
    width = '25%')
#    cb = on_year_change)


user_month_choice = Dropdown(
    name = 'Comparison Month',
    value = comp_months_names[0],
    options = comp_months_names,
    width = '25%')

user_version_choice = Dropdown(
    name = 'MODGRNLD Data Version',
    value = version_options[0],
    options = version_options,
    width = '25%')

exe_button = widgets.Button(
    description = 'Run Analysis',
    disabled = False,
    button_style = 'success'
)
exe_button.on_click(execute)

confirmation_button = widgets.Button(
    description = 'Continue Analysis',
    disabled = False,
    button_style = 'info')
#confirmation_button.on_click(execute_after_confirmation)

cancel_button = widgets.Button(
    description = 'Cancel Analysis',
    disabled = False,
    button_style = 'danger')
cancel_button.on_click(execute_cancelled)

progress_bar = widgets.IntProgress(
    value = 0,
    min = 0,
    max = 10,
    description = 'Loading: ',
    bar_style = 'info',
    layout = {'visibility':'hidden'})
button_and_progress = widgets.HBox([exe_button,progress_bar])

In [6]:
text_output = widgets.Output(layout = {'width':'50%','border':'2px solid black'})
modgrnld_output = widgets.Output(layout = {'width':'35%','border':'2px solid black'})
surfacetemp_output = widgets.Output(layout = {'width':'50%','border':'2px solid black'}) 
stats_output = widgets.Output(layout = {'width':'35%','border':'2px solid black'})

text_output.layout.visibility = 'hidden'
modgrnld_output.layout.visibility = 'hidden'
surfacetemp_output.layout.visibility = 'hidden'
stats_output.layout.visibility = 'hidden'

text = ' Plotting Information '
text_label = widgets.HTML(value = f"<b>{text}</b>")
text = ' MODGRNLD Data: Number of Days used in Monthly Average Calculation '
modgrnld_label = widgets.HTML(value = f"<b>{text}</b>") #, layout = {'border':'1px solid black','width':'auto'})
text = " SURFACE TEMPERATURE DATA: "
surftemp_label = widgets.HTML(value = f"<b>{text}</b>") #, layout = {'border':'1px solid black','width':'auto'})
text = ' STATISTICS AND DIFFERENCE HISTOGRAM: '
stats_label = widgets.HTML(f"<b>{text}</b>") #, layout = {'border':'1px solid black','width':'auto'})

text_label.layout.visibility = 'hidden'
modgrnld_label.layout.visibility = 'hidden'
surftemp_label.layout.visibility = 'hidden'
stats_label.layout.visibility = 'hidden'

box_layout = widgets.Layout(display = 'flex',
                   flex_flow = 'row',
                   align_items = 'stretch',
                   width = 'auto')

box2_layout = widgets.Layout(display = 'flex',
                            flex_flow = 'column',
                            align_items = 'stretch')
output_box1 = widgets.Box(children=[text_output,modgrnld_output], layout = box_layout)
output_box2 = widgets.Box(children=[surfacetemp_output,stats_output],layout=box_layout)
out_box = widgets.Box(children = [output_box1, output_box2], layout = box2_layout)

output_box1.layout.visibility = 'hidden'
output_box2.layout.visibility = 'hidden'
out_box.layout.visibility = 'hidden'

control_panel = widgets.VBox(children = [user_variable_choice, user_year_choice, user_month_choice, user_version_choice, button_and_progress], layout = {'width':'100%'})
login_panel = widgets.Output()
control_and_login = widgets.HBox(children = [control_panel, login_panel])

ui_display = widgets.Output()

In [7]:
#EARTHDATA LOGIN
#mylogin = NASALogin()
#mylogin.view()

In [8]:
display(ui_display)
ui_display.clear_output()
with ui_display:
    display(control_and_login)    
    
display(out_box)

Output()

Box(children=(Box(children=(Output(layout=Layout(border='2px solid black', visibility='hidden', width='50%')),…

<a id = 'MODGRNLD_cell'></a>
**MODGRNLD Data**

This multilayer data set includes standard MODIS Collection 6.1 ice surface temperature (IST) and derived melt map, as well as MODIS Collection 6.0 albedo and water vapor for Greenland, at a spatial resolution of 0.78 km. These fields enable the relationship between IST and surface melt to be evaluated by researchers studying surface changes on the Greenland ice sheet. Water vapor is included to assist with evaluating the accuracy of the IST data and the model output. Also included is an ice mask and a basins mask for delineating drainage basins in Greenland.

The current MODGRNLD data available is part of MODGRNLD, Version 1. This is the initial release of the MODGRNLD data set. 

MODGRNLD Data used in this tool can be found at: https://nsidc.org/data/MODGRNLD

<a id = 'MERRA2_cell'></a>
**MERRA-2 Data**

The MERRA-2 data product that we will use is the statistics of the single-level-variable (slv) collection, statD_2d_slv_Nx (https://dx.doi.org/10.5067/9SC1VNTWGWV3). These products contain daily averages of the temperature at various heights above the surface. We will use the 2-m temperature contained in the variable **T2MEAN**. We must download the daily-averaged MERRA-2 data and then average over the month.

MERRA-2 Data used in this tool can be found at: https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2

## Notes 
### <span style='background :yellow'><span style='color:Red'>CAUTION:</span></span> Check the number of days that were used in the monthly average for the MODGRNLD product.
The product is an "opportunistic" average in that it calculates the average monthly temperature within a given pixel using any available daily observations. The spatiotemporal distribution of daily observations might be heterogeneous across the ice sheet and this needs to be taken into account when interpreting model-observation differences. Here, we plot the number of days that were used to calculate the monthly average, giving one indicator of how representative the calculated monthly average is of the true monthly average. To investigate whether the calculated monthly average is biased in time, one would need to download the daily data products and identify the days for which data exists for each pixel.

This plot is diagnostic and is displayed for the user's interpretation.

### <span style='background :yellow'><span style='color:Red'>NOTE:</span></span> Large systematic offset between mean MODIS and MERRA-2 ice sheet temperature may exist!
As shown in Hall et al. (2018), there may be a large discrepancy between measured ice sheet surface temperature from MODIS and modeled temperature from MERRA-2 due to the acquisition times of the MODIS instrument. When comparing a monthly average of available MODIS measurements against an average of <u>all</u> MERRA-2 model output,  this discrepancy may exist. When MERRA-2 hourly data is selected to match the acquisition times of MODIS and then averaged to a monthly mean, this discrepancy may go away.